In [1]:
""" Code taken from:
https://click.endnote.com/viewer?doi=10.1016%2Fj.cpc.2018.02.004&token=WzI1MTkzMTMsIjEwLjEwMTYvai5jcGMuMjAxOC4wMi4wMDQiXQ.Me0dEpnjbzi1YczwpCuoLJWfP9A
"""

## Import packages
from qutip import *
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patheffects as pe
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
import math
%matplotlib inline
from matplotlib import rc
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
font_size = 14


In [ ]:
## Dimensional operators
cav_dim = 50 # types of photons there can exist (thermal or participaitn gin excitation)
atm_dim = 2 # energy levels in atom, 2 for TLS
I_cav = qeye(cav_dim) # identity matrix matching dimension of radiation field
I_atom = qeye(atm_dim) # identity matrix matching dimension of spin system

## Atomic spin system operators
sigmap = tensor(sigmap(), I_cav)
sigmam = tensor(sigmam(), I_cav)
sigmaz = tensor(sigmaz(), I_cav)

## Photon operators
adag = tensor(I_atom, create(cav_dim)) # order of arguments is important and is consistant with the spin operators                
a = tensor(I_atom, destroy(cav_dim))

## Physical constants
w = 2 * np.pi * 1.45e3 # cavity frequency with vacuum field energy set to zero (zero-point energy) (MHz)
g = 1.5 #2 * np.pi * 1.1 # test spin-photon couling strength ~ np.sqrt(tls_num)*g, experimetnally it's 1.1 though. 

## Initial quantum states
psi_atm = basis(2, 1) # atom initially stat in ground state
psi_phot_fock = fock(cav_dim, 1) # exactly 16 photons in the cavity out of a possible 40
psi_phot_fock2 = fock(cav_dim, 4)
psi_phot_fock3 = fock(cav_dim, 20)

psi_phot_coher = coherent(cav_dim, np.sqrt(1)) # exactly 16 photons in the cavity out of a possible 40
psi_phot_coher2 = coherent(cav_dim, np.sqrt(4))
psi_phot_coher3 = coherent(cav_dim, np.sqrt(20))



psi0_fock = tensor(psi_atm, psi_phot_fock)
psi0_fock2 = tensor(psi_atm, psi_phot_fock2)
psi0_fock3 = tensor(psi_atm, psi_phot_fock3)

psi0_coher = tensor(psi_atm, psi_phot_coher)
psi0_coher2 = tensor(psi_atm, psi_phot_coher2)
psi0_coher3 = tensor(psi_atm, psi_phot_coher3)



## Hamiltonian
H = g*(sigmap*a + sigmam*adag)

## Collapse operators
c_ops = []

Kc = 0.0
Ks = 0.0
gamma = 0.

c_ops.append(np.sqrt(Kc) * a)
c_ops.append(np.sqrt(Ks) * sigmaz)
c_ops.append(np.sqrt(gamma) * sigmam)

## simulation duration and master equation solver
time = 40
steps = 2000
tlist = np.linspace(0, time, steps)
result_fock = mesolve(H, psi0_fock, tlist, c_ops, [adag*a, sigmaz, sigmap*sigmam, adag*sigmam], options=Options(nsteps=10000))
result_fock2 = mesolve(H, psi0_fock2, tlist, c_ops, [adag*a, sigmaz, sigmap*sigmam, adag*sigmam], options=Options(nsteps=10000))
result_fock3 = mesolve(H, psi0_fock3, tlist, c_ops, [adag*a, sigmaz, sigmap*sigmam, adag*sigmam], options=Options(nsteps=10000))


result_coher = mesolve(H, psi0_coher, tlist, c_ops, [adag*a, sigmaz, sigmap*sigmam, adag*sigmam], options=Options(nsteps=10000))
result_coher2 = mesolve(H, psi0_coher2, tlist, c_ops, [adag*a, sigmaz, sigmap*sigmam, adag*sigmam], options=Options(nsteps=10000))
result_coher3 = mesolve(H, psi0_coher3, tlist, c_ops, [adag*a, sigmaz, sigmap*sigmam, adag*sigmam], options=Options(nsteps=10000))



In [ ]:
## Visualization

fig, ax = plt.subplots(figsize=(5,4))
plt.figure(1)
plt.plot(tlist, result_fock.expect[0], color='r', linestyle='-', linewidth=0.5, alpha=0.2, zorder=0, label='Fock')
plt.plot(tlist, result_coher.expect[0], color='magenta', linestyle='-', path_effects=[pe.Stroke(linewidth=2, foreground='w'), pe.Normal()], linewidth=1.5, zorder=0, label='Coherent')
plt.ylim([-0.07, 1.07])
plt.ylabel(r'$\langle\ a^\dag a \ \rangle$', fontsize = 16)
plt.xlabel('Time (μs)', labelpad = 10, fontsize = 16)
plt.xticks(fontsize = 14)
plt.yticks(fontsize = 14)
ax.xaxis.set_major_locator(MultipleLocator(5))
ax.yaxis.set_minor_locator(MultipleLocator(0.1))
ax.tick_params(which='minor', length=3)
plt.legend(fancybox=False, edgecolor='w', fontsize=13, loc="lower right", ncol=2)
plt.tight_layout()
plt.savefig('coherent vs fock 1.png', format = 'png', dpi = 900) 

fig, ax = plt.subplots(figsize=(5,4))
plt.figure(2)
plt.plot(tlist, result_fock2.expect[0], color='r', linestyle='-', linewidth=0.5, alpha=0.2, zorder=0, label='Fock')
plt.plot(tlist, result_coher2.expect[0], color='magenta', linestyle='-', path_effects=[pe.Stroke(linewidth=2, foreground='w'), pe.Normal()], linewidth=1.5, label='Coherent')
plt.ylim([2.97, 4.07])
plt.ylabel(r'$\langle\ a^\dag a \ \rangle$', fontsize = 16)
plt.xlabel('Time (μs)', labelpad = 10, fontsize = 16)
plt.xticks(fontsize = 14)
plt.yticks(fontsize = 14)
ax.xaxis.set_major_locator(MultipleLocator(5))
ax.yaxis.set_minor_locator(MultipleLocator(0.1))
ax.tick_params(which='minor', length=3)
plt.legend(fancybox=False, edgecolor='w', fontsize=13, loc="lower right", ncol=2)
plt.tight_layout()
plt.savefig('coherent vs fock 2.png', format = 'png', dpi = 900) 

fig, ax = plt.subplots(figsize=(5.1,4))
plt.figure(3)
plt.plot(tlist, result_fock3.expect[0], color='r', linestyle='-', linewidth=0.5, alpha=0.2, zorder=0, label='Fock')
plt.plot(tlist, result_coher3.expect[0], color='magenta', linestyle='-', path_effects=[pe.Stroke(linewidth=2, foreground='w'), pe.Normal()], linewidth=1.5, label='Coherent')
#plt.ylim([8.97, 10.07])
plt.ylabel(r'$\langle\ a^\dag a \ \rangle$', fontsize = 16)
plt.xlabel('Time (μs)', labelpad = 10, fontsize = 16)
plt.xticks(fontsize = 14)
plt.yticks(fontsize = 14)
ax.xaxis.set_major_locator(MultipleLocator(5))
ax.yaxis.set_minor_locator(MultipleLocator(0.1))
ax.tick_params(which='minor', length=3)
plt.legend(fancybox=False, edgecolor='w', fontsize=13, loc="lower right", ncol=2)
plt.tight_layout()
plt.savefig('coherent vs fock 3.png', format = 'png', dpi = 900) 




In [ ]:
N = 50
rho_coherent1 = coherent_dm(10, np.sqrt(3))
rho_coherent2 = coherent_dm(10, np.sqrt(3))
rho_coherent3 = coherent_dm(11, np.sqrt(3))
rho_thermal1 = thermal_dm(N, 10)
rho_thermal2 = thermal_dm(N, 20)
rho_thermal3 = thermal_dm(N, 30)
rho_fock1 = fock_dm(N, 10)
rho_fock2 = fock_dm(N, 20)
rho_fock3 = fock_dm(N, 30)
fig, axes = plt.subplots(1, 3, figsize=(11.5,3))
axes[0].bar(np.arange(0, 10), rho_coherent1.diag(), color='cornflowerblue', linewidth=0.8, edgecolor='blue', alpha=1)
axes[0].bar(np.arange(0, 10), rho_coherent2.diag(), color='mediumpurple', linewidth=0.6, edgecolor='indigo', alpha=0.8)
axes[0].bar(np.arange(0, 11), rho_coherent3.diag(), color='tomato', linewidth=0.4, edgecolor='red', alpha=0.7)
axes[0].set_title("Coherent state", fontsize=14)
axes[0].set_xlim([-.5, N+0.5])
axes[0].set_ylabel('Occupational Probability, \n $P_N$', labelpad = 15, fontsize=14)
axes[0].tick_params(axis='both', which='major', labelsize=12)
axes[0].set_xticks(np.arange(0, N+1, 10))
axes[1].bar(np.arange(0, N), rho_thermal1.diag(), color='cornflowerblue', linewidth=0.8, edgecolor='blue', alpha=1)
axes[1].bar(np.arange(0, N), rho_thermal2.diag(), color='mediumpurple', linewidth=0.6, edgecolor='indigo', alpha=0.8)
axes[1].bar(np.arange(0, N), rho_thermal3.diag(), color='tomato', linewidth=0.4, edgecolor='red', alpha=0.7)
axes[1].set_title("Thermal state", fontsize=14)
axes[1].set_xlim([-.5, N+0.5])
axes[1].set_xlabel('Photon number, $N$', labelpad = 10, fontsize=14)
axes[1].set_xticks(np.arange(0, N+1, 10))
axes[1].tick_params(axis='both', which='major', labelsize=12)
axes[2].bar(np.arange(0, N), rho_fock1.diag(), color='cornflowerblue', linewidth=0.8, edgecolor='blue', alpha=1)
axes[2].bar(np.arange(0, N), rho_fock2.diag(), color='mediumpurple', linewidth=0.6, edgecolor='indigo', alpha=0.8)
axes[2].bar(np.arange(0, N), rho_fock3.diag(), color='tomato', linewidth=0.4, edgecolor='red', alpha=0.7)
axes[2].set_title("Fock state", fontsize=14)
axes[2].set_xlim([-.5, N+0.5])
axes[2].set_xticks(np.arange(0, N+1, 10))
axes[2].tick_params(axis='both', which='major', labelsize=12)

colors = {r'$\langle N \rangle = 10$':'cornflowerblue', r'$\langle N \rangle = 20$':'mediumpurple', r'$\langle N \rangle = 30$':'tomato'}         
labels = list(colors.keys())
handles = [plt.Rectangle((0,0),1,1, color=colors[label]) for label in labels]
axes[1].legend(handles, labels, frameon=False, loc='upper right', fontsize=11)


plt.tight_layout()
#plt.savefig('light.png', format = 'png', dpi = 900) 

plt.show()




In [ ]:
coherent(10, np.sqrt(0))

In [ ]:
coherent(10, np.sqrt(1))

In [ ]:
coherent(10, np.sqrt(2))

In [ ]:
coherent(10, np.sqrt(3))

In [ ]:
destroy(40)*coherent(40, np.sqrt(3))

In [ ]:
coherent(40, np.sqrt(1))

In [ ]:
coherent(4, np.sqrt(2))

In [ ]:
coherent(4, np.sqrt(3))

In [9]:
r = ket2dm(np.sqrt(0.8)*basis(2,1) + np.sqrt(0.2)*basis(2,0))

In [10]:
0.5*r + 0.5*sigmaz()*r*sigmaz()

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0.2 0. ]
 [0.  0.8]]

In [ ]:
psi_atm = np.sqrt(0.2)*basis(2, 0) + np.sqrt(0.8)*basis(2, 1)
rho_atm = ket2dm(psi_atm)
psi_phot = np.sqrt(0.2)*fock(2, 0) + np.sqrt(0.8)*fock(2, 1)
rho_phot = ket2dm(psi_phot)
rho0 = tensor(rho_atm, rho_phot)
rho0

In [ ]:
basis(2,0) * basis(2,1).dag()

In [ ]:
sigmap()

In [ ]:
np.sqrt(0.6)*basis(2,1) + np.sqrt(0.4)*basis(2,0)

In [ ]:
## http://qutip.org/docs/4.1/guide/guide-visualization.html